# Analyse de l'Évolution de la Température Moyenne en France : 1950-2023


Ce notebook présente une analyse détaillée de l'évolution de la température moyenne en France sur une période de plus de 70 ans, allant de 1950 à 2023. Les données utilisées proviennent d'un fichier CSV disponible sur le site data.gouv.fr, qui contient des relevés météorologiques quotidiens de diverses stations en France. L'URL source du fichier est : https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_62_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des informations sur la température, les précipitations, la vitesse du vent, etc.


L'objectif de cette analyse est de calculer la moyenne mensuelle de la température moyenne (TM) et son lissage sur 120 mois glissants, afin de mettre en évidence les tendances climatiques à long terme. Cette analyse s'inscrit dans le thème de la météorologie et du climat, et vise à contribuer à une meilleure compréhension des évolutions climatiques en France.


## Méthodologie


La méthodologie utilisée dans cette analyse consiste à importer les données dans un dataframe, puis à les traiter à l'aide de requêtes SQL via la bibliothèque DuckDB. Les données sont d'abord filtrées pour ne conserver que les enregistrements où la température moyenne est non nulle et de qualité égale à '1'. Ensuite, la moyenne mensuelle de la température moyenne est calculée, puis lissée sur 120 mois glissants à l'aide d'une fenêtre mobile. Les résultats sont finalement visualisés à l'aide de la bibliothèque Bokeh, qui permet de créer des graphiques interactifs.


Les sections suivantes présentent les résultats de l'analyse et les visualisations associées.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_62_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH monthly_avg AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 6) AS INTEGER) AS yyyymm,
    AVG(CAST("TM" AS DOUBLE)) AS avg_tm
  FROM 
    loaded_dataset
  WHERE 
    "TM" IS NOT NULL AND "QTM" = '1'
  GROUP BY 
    CAST(SUBSTRING("AAAAMMJJ", 1, 6) AS INTEGER)
),
smoothed_avg AS (
  SELECT 
    yyyymm,
    avg_tm,
    AVG(avg_tm) OVER (ORDER BY yyyymm ROWS BETWEEN 119 PRECEDING AND CURRENT ROW) AS smoothed_tm
  FROM 
    monthly_avg
)
SELECT 
  yyyymm,
  avg_tm,
  CASE 
    WHEN ROW_NUMBER() OVER (ORDER BY yyyymm) >= 120 THEN smoothed_tm
    ELSE NULL
  END AS smoothed_tm
FROM 
  smoothed_avg
ORDER BY 
  yyyymm """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 877 lignes


,yyyymm,avg_tm,smoothed_tm
0,195001,3.690323,NaN
1,195002,7.178571,NaN
2,195003,8.119355,NaN
3,195004,8.786667,NaN
4,195005,12.387097,NaN


## 📈 Visualisation

La bibliothèque principale utilisée est Bokeh, qui est adaptée pour créer des visualisations interactives et dynamiques. Bokeh est particulièrement utile ici pour représenter l'évolution temporelle de la température moyenne et son lissage, avec des fonctionnalités telles que le survol et le zoom. Cela permet une exploration détaillée des données.

In [4]:
import pandas as pd
import duckdb as ddb
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Blues9
import pandas as pd
import numpy as np

df['date'] = pd.to_datetime(df['yyyymm'], format='%Y%m')
source = ColumnDataSource(data=dict(
    date=df['date'],
    avg_tm=df['avg_tm'],
    smoothed_tm=df['smoothed_tm']
))
moyenne_totale = df['avg_tm'].mean()

dataviz = figure(title="Évolution de la Température Moyenne et Lissage sur 120 Mois",
                 x_axis_type='datetime',
                 width=900, height=450,
                 toolbar_location='above',
                 tools='pan,box_zoom,wheel_zoom,reset',
                 x_axis_label='Mois', y_axis_label='Température (°C)',
                 background_fill_color='#fafafa')

dataviz.grid.grid_line_color = 'white'
dataviz.axis.axis_line_color = '#555555'
dataviz.axis.major_tick_line_color = '#555555'

dataviz.line('date', 'avg_tm', source=source, color='#c8c8c8', line_width=1.8, alpha=0.7)
dataviz.line('date', 'smoothed_tm', source=source, color='#e31a1c', line_width=2.5)
dataviz.line(df['date'], [moyenne_totale]*len(df), color='#1f77b4', line_width=1.5, line_dash='4 4')

hover = HoverTool(tooltips=[
    ("Mois", "@date{%Y-%m}"),
    ("TM brute", "@avg_tm{0.2f} °C"),
    ("Lissage 120 mois", "@smoothed_tm{0.2f} °C")
], formatters={'@date': 'datetime'}, mode='vline')
dataviz.add_tools(hover)
dataviz.y_range.range_padding = 0.05
dataviz

figure(id='p1006', ...)

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250801_214221.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250801_214221.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Bokeh. Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250801_214221.html
--> Création du marqueur de capture d'écran.


C:\Users\ami\AppData\Local\Temp\ipykernel_16956\886436290.py:67: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
